this script was created per Rich's request. Goal is to calculate total volume of water withdrawal in 2014


In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2017M11D24', 'UTC 13:15')


'2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
import ee
import pandas as pd
import os

In [3]:
ee.Initialize()

In [4]:
SCRIPT_NAME = "Y2017M11D14_RH_Rich_PCR_Sums_V01"
OUTPUT_VERSION =2 

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/%s/output/" %(SCRIPT_NAME)

OUTPUT_FILE_NAME = "Y2017M11D14_RH_Rich_PCR_Sums_V%0.2d" %(OUTPUT_VERSION)

In [5]:
!rm -r {EC2_OUTPUT_PATH}

!mkdir -p {EC2_OUTPUT_PATH}

In [6]:
year = 2014
timeFrame = "year" #annual


In [7]:
sectors = ["Dom","Ind","Irr","Liv"]

In [8]:
demandTypes = ["WW","WN"]

In [9]:
geometry = ee.Geometry.Polygon(coords=[[-180.0, -90.0], [180,  -90.0], [180, 90], [-180,90]], proj= ee.Projection('EPSG:4326'),geodesic=False )

In [10]:
scale = ee.Image(ee.ImageCollection("projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PDomWN_year_millionm3_5min_1960_2014").first()).projection().nominalScale().getInfo()

In [11]:
print(scale)

9276.62397058


In [12]:
df = pd.DataFrame()
fc = ee.FeatureCollection(ee.Feature(None,{}))
for year in range(1960,2014+1):
    
    print(year)
    for sector in sectors:
        for demandType in demandTypes:
            indicator = "%s%s" %(sector,demandType)
            keyName = "%s%sY%0.2d" %(sector,demandType,year)
            filePath = "projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_P%s_year_millionm3_5min_1960_2014" %(indicator)
            ic = ee.ImageCollection(filePath)
            imageYear = ee.Image(ic.filter(ee.Filter.eq("year",year)).first())
            sumYear = imageYear.reduceRegion(
                geometry= geometry,
                reducer= ee.Reducer.sum(),
                scale= scale,
                maxPixels= 1e10
            )
            df.at[year, indicator] = sumYear.get("b1").getInfo()
            
               
    

1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


## Add runoff data

In [13]:
icRunoff = ee.ImageCollection("projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_year_myear_5min_1958_2014")

In [14]:
imageArea = ee.Image("projects/WRI-Aquaduct/PCRGlobWB20V07/area_5min_m2V11")

In [15]:
def fluxToVolume(image):
    image = ee.Image(image)
    newImage = image.multiply(imageArea).divide(1e6)
    newImage = newImage.copyProperties(image)
    newImage = newImage.set("units","millionm^3")
    return newImage

In [16]:
icRunoffMillionm3 = ee.ImageCollection(icRunoff.map(fluxToVolume))

In [17]:
ic = icRunoffMillionm3
for year in range(1960,2014+1):
    print(year)    
    imageYear = ee.Image(ic.filter(ee.Filter.eq("year",year)).first())
    sumYear = imageYear.reduceRegion(
                geometry= geometry,
                reducer= ee.Reducer.sum(),
                scale= scale,
                maxPixels= 1e10
            )
    df.at[year, "localRunoff"] = sumYear.get("b1").getInfo()
    

1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


In [18]:
df.head()

,DomWW,DomWN,IndWW,IndWN,IrrWW,IrrWN,LivWW,LivWN,localRunoff
1960,82829.934120,53617.348704,350122.827989,79004.246007,1.487135e+06,549676.079681,10591.382454,10591.382454,3.892361e+07
1961,84804.556401,54441.266194,351414.480211,79179.859626,1.460775e+06,544174.653526,10679.352846,10679.352846,4.059177e+07
1962,87361.339997,56054.475729,356198.304181,82715.303385,1.514793e+06,561582.611032,10796.260742,10796.260742,4.000380e+07
1963,90545.782340,58139.043652,361817.260393,84065.468090,1.551366e+06,573027.937917,10993.763545,10993.763545,3.897198e+07
1964,94530.612240,60370.855740,370592.055793,83113.262957,1.508349e+06,561405.218008,11259.557907,11259.557907,3.928536e+07


In [19]:
df.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".csv"))

In [20]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M11D14_RH_Rich_PCR_Sums_V01/output/Y2017M11D14_RH_Rich_PCR_Sums_V02.csv to s3://wri-projects/Aqueduct30/processData/Y2017M11D14_RH_Rich_PCR_Sums_V01/output/Y2017M11D14_RH_Rich_PCR_Sums_V02.csv


In [21]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:08:26.675549
